In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
df=pd.read_excel("Data%20scoring%20zypl%20aug%202024.xlsx", sheet_name="Sheet1")
df

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.lower().str.replace(',','').str.replace('.','')
df.columns

In [ ]:
df=df.drop(["nr", "nr_credit", "date_maturity_plan", "date_repaid_de_facto",
            "restructured","other_securities","working_capital_total_mdl","fixed_assets_total_mdl",
            "debts_to_suppliers_mdl","seasonal_activity/sales", "currency", "adress","administration_fee","product"],axis=1)
df.columns

In [ ]:
df["experience_in_business_years"] = df["experience_in_business_years"].astype(str)
print(df["experience_in_business_years"].unique())

df["experience_in_business_years"] = df["experience_in_business_years"].str.replace('+', '').str.replace('luni', '').astype(float)

print(df["experience_in_business_years"].unique())
print(df.shape)


In [ ]:
df["other_debts_to_banks_non_mfis_mdl"]=df["other_debts_to_banks_non_mfis_mdl"].astype(str).str.replace(',', '.')

In [ ]:
cat_columns=df.select_dtypes(include="object").columns

for col in cat_columns:
    if df[col].isna().sum()>0:
        print(f"{col}: {df[col].isna().sum()}")

In [ ]:
nan_counts=df.isna().sum()
nan_counts=nan_counts[nan_counts>0]
print(nan_counts.dtypes)
nan_counts

In [ ]:
df["grace_period"]=df["grace_period"].replace("nu", "0")
df["grace_period"]=df["grace_period"].astype(float)
df["grace_period"].value_counts()

In [ ]:
print(df["social_status"].value_counts())
replacements = {
    "В браке": "married",
    "Не в браке": "single",
    "Сожительство": "cohabitation",
    "В разводе": "divorced"
}

df['social_status'] = df['social_status'].replace(replacements)
df['social_status'].value_counts()

In [ ]:
print(df["background"].value_counts())

replacements = {
    "Школа": "school",
    "Пост лицейское": "post-secondary",
    "Высшее": "higher",
    "Лицей": "lyceum"
}

df['background'] = df['background'].replace(replacements)
df['background'].value_counts()

In [ ]:
print(df["gender"].value_counts())
df["gender"] = df["gender"].apply(lambda x: "male" if x == "Мужской" else "female")
df["gender"].value_counts()

In [ ]:
print(df["economic_sector"].value_counts())

replacements = {
    "СЕЛЬСКОЕ ХОЗЯЙСТВО": "agriculture",
    "ТОРГОВЛЯ": "commerce",
    "УСЛУГИ": "services",
    "ДРУГОЕ": "other",
    "ПРОИЗВОДСТВО/ПРОМЫШЛЕННОСТЬ": "industry"
}

df["economic_sector"]=df["economic_sector"].replace(replacements)
df["economic_sector"].value_counts()


In [ ]:
df=df.applymap(lambda x: x.lower().replace(' ','_').replace('-','_') if isinstance(x, str) else x)

In [ ]:
arr= df.loc[df["status"] == "current", "maxim_days_in_arrears_at_mk"]
arr.value_counts()

In [ ]:
print(df["status"].value_counts())

for index,row in df.iterrows():
    if row["status"]=="repaid":
        df.loc[index, "target"]=1
    elif (row["status"]=="overdue" or row["status"]=="current") and row["maxim_days_in_arrears_at_mk"]<30:
        df.loc[index, "target"]=1
    elif (row["status"]=="overdue" or row["status"]=="current") and row["maxim_days_in_arrears_at_mk"]>=30:
        df.loc[index, "target"]=0
    else:
        df.loc[index, "target"]=0

df["target"].value_counts()

In [ ]:
df=df.dropna(how="all", axis=0)
df=df.dropna(how="all", axis=1)

In [ ]:
#finding columns that have identical values
identical_columns = df.columns[df.nunique() == 1]
zero_columns = df.columns[(df == 0).all()]

print(zero_columns)
identical_columns

In [ ]:
#Removing identical columns
df=df.drop(identical_columns, axis=1)

In [ ]:
cols=[col for col in df.columns if len(col)>20]

col_names=df.columns
new_col_names=[]

for col in col_names:
    col_list=col.split('_')
    if len(col_list)>5:
        col="_".join(col_list[:5])
    new_col_names.append(col)

df.columns=new_col_names
df.columns